In [2]:
import pandas as pd
from datetime import datetime
from datetime import timezone
from sklearn.metrics.pairwise import cosine_similarity
import datetime
import boto3
import pandas as pd
import numpy as np
import requests
import numpy as np

rec = pd.read_csv("dataset.csv")
rec.sort_values(by=["customer_id", "joindate"], inplace=True)
df = rec.head(200000)
df.to_csv("recommendation_v4.csv")
t = pd.read_csv("recommendation_v4.csv")
col = t.columns.to_list()
col = col[1:]
t = t[col]
col = t.columns.to_list()
col = col[:17]
customer = t[col]
customer.to_csv("customers_v2.csv")
cdata = customer[['customer_id', 'email']].drop_duplicates()
cdata.to_csv("c_auth.csv", index=False)
col = t.columns.to_list()
col = col[6:7] + col[:2] + col[17:]
interaction = t[col]
col = interaction.columns.to_list()
products = col[3:]
tmp = pd.melt(interaction, id_vars=["customer_id", "joindate"], value_vars=products, var_name="item_id")
tmp2 = tmp["value"] == 1
interaction = tmp[tmp2]
interaction = interaction.drop(columns="value")
interaction = interaction.drop_duplicates(subset=["customer_id", "item_id"], keep="first")
interaction.sort_values(by=["customer_id", "joindate"], inplace=True)
interaction["customer_id"] = interaction["customer_id"].apply(lambda x: str(x))
interaction["joindate"] = pd.to_datetime(interaction["joindate"])
interaction["joindate"] = interaction["joindate"].apply(lambda x: (x - datetime.datetime(1970, 1, 1)).total_seconds())
col = interaction.columns.to_list()
col = col[0:1] + col[2:3] + col[1:2]
interaction = interaction[col]
interaction.to_csv("interaction_v3.csv", index=False)


def item_based_recommendations(interaction_df, user_id, top_n=2):
    """Recommends items based on item-item similarity."""

    user_interactions = interaction_df[interaction_df["customer_id"] == user_id]
    if user_interactions.empty:
        return "User not found or no interactions"

    user_items = user_interactions["item_id"].unique()
    all_items = interaction_df["item_id"].unique()

    item_matrix = pd.crosstab(interaction_df["customer_id"], interaction_df["item_id"]).astype(int)
    item_similarity = cosine_similarity(item_matrix.T)  # transposed for item similarity

    similarities = {}
    for item in user_items:
        item_index = np.where(item_matrix.columns == item)[0][0]
        similar_items = np.argsort(item_similarity[item_index])[::-1][1:top_n + 1]  # get the top n similar items.
        for similar_item_index in similar_items:
            similar_item = item_matrix.columns[similar_item_index]
            if similar_item not in user_items:  # ensure the user hasn't already seen the item.
                if similar_item not in similarities:
                    similarities[similar_item] = 0
                similarities[similar_item] += item_similarity[item_index, similar_item_index]

    recommended_items = sorted(similarities, key=similarities.get, reverse=True)[:top_n]
    return recommended_items, user_items  # Return also the user_items.


# Example Usage
user_id_to_recommend = "1004158"  # Replace with a valid user ID from your data
recommendations, user_items = item_based_recommendations(interaction, user_id_to_recommend)

print(f"User's current products: {user_items}") # Print the user's current products.
print(f"Users also like : {recommendations}")

User's current products: ['CheckingAccount' 'CreditCard' 'HomeLoan' 'RetirementAccount'
 'MoneyMarketAccount' 'Mortgage' 'DirectDeposit' 'BillPay' 'PersonalLoan'
 'BrokerageAccount' 'OverdraftProtection' 'MoneyTransfer' 'DirectDebit'
 'SafeDepositBox']
Users also like : ['TrustAccount', 'AutoLoan']
